# The Constellation Wizard requires a STK Scenario to be open

Simply run the cell below and the constelation wizard will appear

In [1]:
from tkinter import Tk
from tkinter.ttk import *
from tkinter import W
from tkinter import E
from tkinter import scrolledtext
from tkinter import INSERT
from tkinter import END
from tkinter import IntVar
from tkinter import messagebox
from DeckAccessReaderGUI import *
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
from shutil import copyfile
from comtypes.client import CreateObject
from comtypes.client import GetActiveObject
from comtypes.gen import STKObjects



# Define window layout
window = Tk()
window.title('Constellation Wizard')
window.geometry('587x510')
cwd = os.getcwd()
cwdFiles = cwd+'\\Files'
window.iconbitmap(cwdFiles+'\\Misc\\'+'ConstellationWizardIcon.ico')


# # Configure Style
Style().theme_use('vista')

# # fonts for all widgets
# window.option_add("*Font", "calabri 9")


#########################################  Col0   ########################################################
width = 35
padx = 3
pady = 1
column=0
row = 1
# Connect to STK
try:
    root = ConnectToSTK(version=12)
    startTime = root.CurrentScenario.QueryInterface(STKObjects.IAgScenario).StartTime
    stopTime = root.CurrentScenario.QueryInterface(STKObjects.IAgScenario).StartTime+3600
except:
    res = messagebox.askyesno('Constellation Wizard','Failed to connect to a scenario.\nIs a scenario in STK open?')
    if res == True:
        try:
            root = ConnectToSTK(version=12)
            startTime = root.CurrentScenario.QueryInterface(STKObjects.IAgScenario).StartTime
            stopTime = root.CurrentScenario.QueryInterface(STKObjects.IAgScenario).StartTime+3600
        except:
            window.quit()
            window.destroy()
    else:
        window.quit()
        window.destroy()
            


def createConMsgBox():
    res = messagebox.askyesno('Constellation Wizard',txt.get().replace(' ','-')+'.tce will be created and overwrite any existing file.\nThis may take a while if there are many satellites in the scenario.\nContinue?')
    if res == True:
        CreateConstellation(root,txt,txtBox,ssc=00000)

btnCreateCon = Button(window,width=width,text='Create Constellation From STK',command=lambda: createConMsgBox())
btnCreateCon.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2)
row += 1

# Load MTO
btnLoadMTO = Button(window,width=width,text='Load Constellation as MTO',command=lambda: LoadMTO(root,txtBox,MTOName = comboCon.get(),timestep=60,color=comboColor.get().lower(),orbitsOnOrOff=onOffStr(),orbitFrame=frameValue()))
btnLoadMTO.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2)
row += 1

# Orbit options
lblFrame = Label(window,text = 'Show Orbits:')
lblFrame.grid(column=column,row=row,padx=padx,pady=pady,sticky=E)

# Checkbox
def onOffStr():
    onOff = showOrbits.get()
    if onOff == 0: 
        onOff = 'off'
    elif onOff == 1:
        onOff = 'on'    
    return onOff

showOrbits = IntVar()
showOrbits.set(0)
checkButton = Checkbutton(window, variable=showOrbits,offvalue=0,onvalue=1)
checkButton.grid(column=column+1,row=row,padx=padx,pady=pady,sticky=W)

row += 1
row += 1

# Run Deck Access
btnDeckAccess = Button(window,width=width,text='Run Deck Access',command=lambda: runDeckAccess(root,txtStart.get(),txtStop.get(),comboCon,comboDA,txtBox,constraintSatName = comboSat.get()))
btnDeckAccess.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2)
row += 1

# Save Deck Access
def saveDA():
    newName = txt.get().replace(' ','-')
    res = messagebox.askyesno('Constellation Wizard',newName+'.tce will be created and overwrite any existing file.\nContinue?')
    if res == True:
        copyfile(cwdFiles+'\\Constellations\\deckAccessTLE.tce', cwdFiles+'\\Constellations\\'+newName+'.tce')
        txtBox.insert(END,'Created: '+txt.get().replace(' ','-')+'.tce\n')
    
btnSave = Button(window,text='Save Deck Access',command=saveDA)
btnSave.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 1,sticky=W+E)
row += 2

# # Load Subset
btnLoadSubset = Button(window,width=width,text='Load Satellites Using Template',command= lambda: LoadSatsFromFileUsingTemplate(root,txtStart.get(),txtStop.get(),comboCon,selected,txtBox,comboSat.get(),color=comboColor.get().lower()))
btnLoadSubset.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2)
row += 2

# Do Analysis
def AddToChain():
    addObj = comboChainCov.get()
    chainName = comboChain.get()
    try:
        chain = root.GetObjectFromPath('*/Chain/'+chainName)
        chain2 = chain.QueryInterface(STKObjects.IAgChain)
        chain2.Objects.Add(addObj)
        txtBox.insert(END,'Added: '+addObj.split('/')[-1]+'\n')
    except:
        txtBox.insert(END,'Failed to Add: '+addObj.split('/')[-1]+'\n')
            
btnAddChain = Button(window,width=width,text='Add To Chain',command=AddToChain)
btnAddChain.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2)
row += 1

# Do Analysis
def computeChain():
    chainName = comboChain.get()
    if root.CurrentScenario.Children.Contains(STKObjects.eChain,chainName):
        chain = root.GetObjectFromPath('*/Chain/'+chainName)
        chain2 = chain.QueryInterface(STKObjects.IAgChain)
        chain2.ClearAccess()
        chain2.ComputeAccess()
        txtBox.insert(END,'Computed: '+chainName+'\n')
    else:
        txtBox.insert(END,'Failed to Compute: '+chainName+'\n')
btnComputeChain = Button(window,text='Compute Chain',command=computeChain)
btnComputeChain.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 1,sticky=W+E)


def removeAssets():
    chainName = comboChain.get()
    if root.CurrentScenario.Children.Contains(STKObjects.eChain,chainName):
        chain = root.GetObjectFromPath('*/Chain/'+chainName)
        chain2 = chain.QueryInterface(STKObjects.IAgChain)
        chain2.Objects.RemoveAll()
        txtBox.insert(END,'Removed Objects: '+chainName+'\n')
    else:
        txtBox.insert(END,'Failed to Removed Objects: '+chainName+'\n') 
btnRemoveChain = Button(window,text='Remove Objects',command=removeAssets)
btnRemoveChain.grid(column=column+1,row=row,padx=padx,pady=pady,columnspan = 1,sticky=W+E)
row += 1

 # Do Analysis
def AddToCoverage():
    addObj = comboChainCov.get()
    covName = comboCov.get()
    if root.CurrentScenario.Children.Contains(STKObjects.eCoverageDefinition,covName):
        cov = root.GetObjectFromPath('*/CoverageDefinition/'+covName)
        cov2 = cov.QueryInterface(STKObjects.IAgCoverageDefinition)
        if cov2.AssetList.CanAssignAsset(addObj):
            cov2.AssetList.Add(addObj)
            txtBox.insert(END,'Added: '+addObj.split('/')[-1]+'\n')
        else:
            txtBox.insert(END,'Already Assigned: '+addObj.split('/')[-1]+'\n')
    else:
            txtBox.insert(END,'Failed to Add: '+addObj.split('/')[-1]+'\n')
        
btnAddCoverage = Button(window,width=width,text='Add To Coverage',command=AddToCoverage)
btnAddCoverage.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2)
row += 1

# Do Analysis
def computeCov():
    covName = comboCov.get()
    if root.CurrentScenario.Children.Contains(STKObjects.eCoverageDefinition,covName):
        cov = root.GetObjectFromPath('*/CoverageDefinition/'+covName)
        cov2 = cov.QueryInterface(STKObjects.IAgCoverageDefinition)
        cov2.ClearAccesses()
        cov2.ComputeAccesses()
        txtBox.insert(END,'Computed: '+covName+'\n')
    else:
        txtBox.insert(END,'Failed to Compute: '+covName+'\n')
btnComputeCoverage = Button(window,text='Compute Coverage',command=computeCov)
btnComputeCoverage.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 1,sticky=W+E)

def removeAssestsCov():
    covName = comboCov.get()
    if root.CurrentScenario.Children.Contains(STKObjects.eCoverageDefinition,covName):
        cov = root.GetObjectFromPath('*/CoverageDefinition/'+covName)
        cov2 = cov.QueryInterface(STKObjects.IAgCoverageDefinition)
        cov2.AssetList.RemoveAll()
        txtBox.insert(END,'Removed Assets: '+covName+'\n')
    else:
        txtBox.insert(END,'Failed to Removed Assets: '+covName+'\n')
btnRemoveCov = Button(window,text='Remove Assets',command=removeAssestsCov)
btnRemoveCov.grid(column=column+1,row=row,padx=padx,pady=pady,columnspan = 1,sticky=W+E)
row += 1
row += 3


txtBox = scrolledtext.ScrolledText(window,width=35,height=10)
txtBox.insert(INSERT,'Connected: '+root.CurrentScenario.InstanceName+'\n')
txtBox.grid(column=column,row=row,padx=padx+0,pady=pady,rowspan=4,columnspan = 3,sticky=W+E)
rowTxt = row


#########################################  Col2   ########################################################
# Labels
width2 = 30
column = 2
row = 1
lblCreateCon = Label(window,text = 'Create/Save Constellation:')
lblCreateCon.grid(column=column,row=row,padx=padx,pady=pady,sticky=E)
row += 1
lblCon = Label(window,text = 'Constellation:')
lblCon.grid(column=column,row=row,padx=padx,pady=pady,sticky=E)
row += 1
# MTO Options
row += 1
lblColor = Label(window,text = 'MTO/Satellite Color:')
lblColor.grid(column=column,row=row,padx=padx,pady=pady,sticky=E)
row +=1
lblDA = Label(window,text = 'Access From:')
lblDA.grid(column=column,row=row,padx=padx,pady=pady,sticky=E)
row += 1
lblStart = Label(window,text = 'Start Time:')
lblStart.grid(column=column,row=row,padx=padx,pady=pady,sticky=E)
row += 1
lblStop = Label(window,text = 'Stop Time:')
lblStop.grid(column=column,row=row,padx=padx,pady=pady,sticky=E)
row += 1
lblSatTemp = Label(window,text = 'Satellite Template:')
lblSatTemp.grid(column=column,row=row,padx=padx,pady=pady,sticky=E)
row += 2
lblSatTemp = Label(window,text = 'Chain/Coverage Object:')
lblSatTemp.grid(column=column,row=row,padx=padx,pady=pady,sticky=E)
row += 1
lblSatTemp = Label(window,text = 'Chain:')
lblSatTemp.grid(column=column,row=row,padx=padx,pady=pady,sticky=E)
row += 1
lblSatTemp = Label(window,text = 'Coverage:')
lblSatTemp.grid(column=column,row=row,padx=padx,pady=pady,sticky=E)
row += 2

#########################################  Col3   ########################################################
column = 3
row=1
# Entry box for Create Constellation
txt = Entry(window,width=width2+3)
txt.delete(0, END)
txt.insert(0, 'NewConstellationName')
txt.grid(column=column, row=row,padx=padx,pady=pady,columnspan=2,sticky=W)
row += 1

# Constellation Options
def updateTCEList():
    tceList = [f.split('.')[0] for f in listdir(cwdFiles+'\\Constellations') if (isfile(join(cwdFiles+'\\Constellations', f))) & (f.split('.')[-1]=='tce' )& (f !='deckAccessTLE.tce')]
    comboCon['values'] = tceList
    
comboCon = Combobox(window,width=width2,state='readonly',postcommand = updateTCEList)
updateTCEList()
comboCon.current(0) # set the selected item
comboCon.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2,sticky=W)
row += 1


# Radio Buttons
def frameValue():
    frame = selectedFrame.get()
    if frame == 0:
        frame = 'Inertial'
    elif frame == 1:
        frame = 'Fixed'
    return frame
    
selectedFrame = IntVar()
selectedFrame.set(0)
radFrame1 = Radiobutton(window,text='Inertial', value=0, variable=selectedFrame)
radFrame2 = Radiobutton(window,text='Fixed', value=1, variable=selectedFrame)
radFrame1.grid(column=column-1,row=row,padx=padx,pady=pady)
radFrame2.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2,sticky=W)
row += 1

# Colors
colorsList = ['Green','Cyan','Blue','Magenta','Red','Yellow','White','Black']
comboColor = Combobox(window,width=width2,state='readonly')
comboColor['values'] = colorsList
comboColor.current(0) # set the selected item
comboColor.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2,sticky=W)
row +=1


# Deck Access Available Objects
def updateAccessList(root):
    objs = deckAccessAvailableObjs(root)
    for ii in range(len(objs)):
        objType = objs[ii].split('/')[-2]
        if objType == 'Sensor':
            objs[ii] = '/'.join(objs[ii].split('/')[-4:])
        else:
            objs[ii] = '/'.join(objs[ii].split('/')[-2:])
    comboDA['values'] = objs
    
comboDA = Combobox(window,width=width2,state='readonly',postcommand = lambda: updateAccessList(root))
updateAccessList(root)
try:
    comboDA.current(0) # set the selected item
except:
    pass
comboDA.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2,sticky=W)
row += 1

# Entry box Times
startTimeUTCG = root.ConversionUtility.ConvertDate('EpSec','UTCG',str(startTime))
txtStart = Entry(window,width=width2+3)
txtStart.delete(0, END)
txtStart.insert(0, startTimeUTCG)
txtStart.grid(column=column,row=row,padx=padx,pady=pady,columnspan=2,sticky=W)
startTime = root.ConversionUtility.ConvertDate('UTCG','EpSec',str(txtStart.get()))
row += 1

stopTimeUTCG = root.ConversionUtility.ConvertDate('EpSec','UTCG',str(stopTime))
txtStop = Entry(window,width=width2+3)
txtStop.delete(0, END)
txtStop.insert(0, stopTimeUTCG)
txtStop.grid(column=column,row=row,padx=padx,pady=pady,columnspan=2,sticky=W)
stopTime = root.ConversionUtility.ConvertDate('UTCG','EpSec',str(txtStop.get()))
row += 1

# Satellite Template
def updateSatList(root): 
    sats = FilterObjectsByType(root,'Satellite',name = '')
    for ii in range(len(sats)):
        sats[ii] = sats[ii].split('/')[-1]
    sats.insert(0,'')
    comboSat['values'] = sats
comboSat = Combobox(window,width=width2,state='readonly',postcommand = lambda: updateSatList(root))
updateSatList(root)
try:
    comboSat.current(0) # set the selected item
except:
    pass
comboSat.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2,sticky=W)
row += 1


# Radio Buttons
selected = IntVar()
selected.set(1)
rad1 = Radiobutton(window,text='Deck Access Only', value=1, variable=selected)
rad2 = Radiobutton(window,text='Entire Constellation', value=2, variable=selected)
rad1.grid(column=column-1,row=row,padx=padx,pady=pady)
rad2.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2,sticky=W)

row += 1

# Deck Access Available Objects

def updateChainCovList(root):
    objs = chainCovAvailableObjs(root)
    for ii in range(len(objs)):
        objSplit = objs[ii].split('/')
        if objSplit[-4] =='Scenario':
            objs[ii] = '/'.join(objSplit[-2:])
        elif objSplit[-4]=='Sensor':
            objs[ii] = '/'.join(objSplit[-6:])
        else:
            objs[ii] = '/'.join(objSplit[-4:])
    comboChainCov['values'] = objs
comboChainCov = Combobox(window,width=width2,state='readonly',postcommand = lambda: updateChainCovList(root))
updateChainCovList(root)
try:
    comboChainCov.current(0) # set the selected item
except:
    pass
comboChainCov.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2,sticky=W)
row += 1

# Chain Template
def updateChainList(root): 
    chains = FilterObjectsByType(root,'Chain',name = '')
    for ii in range(len(chains)):
        chains[ii] = chains[ii].split('/')[-1]
#     chains.insert(0,'')
    comboChain['values'] = chains
comboChain = Combobox(window,width=width2,state='readonly',postcommand = lambda: updateChainList(root))
updateChainList(root)
try:
    comboChain.current(0) # set the selected item
except:
    pass
comboChain.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2,sticky=W)
row += 1

# Chain Coverage
def updateCovList(root): 
    covs = FilterObjectsByType(root,'CoverageDefinition',name = '')
    for ii in range(len(covs)):
        covs[ii] = covs[ii].split('/')[-1]
#     covs.insert(0,'')
    comboCov['values'] = covs
comboCov = Combobox(window,width=width2,state='readonly',postcommand = updateCovList)
updateCovList(root)
try:
    comboCov.current(0) # set the selected item
except:
    pass
comboCov.grid(column=column,row=row,padx=padx,pady=pady,columnspan = 2,sticky=W)
row += 2

# row += 4
            

# Unload Satellites
btnUnload = Button(window,width=15,text='Unload Satellites',command=lambda: UnloadObjs(root,'Satellite',pattern=txtUnload.get()))
btnUnload.grid(column=3,row=rowTxt+0,padx=padx,pady=pady,sticky=W+E)

txtUnload = Entry(window,width=15)
txtUnload.delete(0, END)
txtUnload.insert(0, 'tle-*')
txtUnload.grid(column=4,row=rowTxt+0,padx=padx,pady=pady,columnspan = 1,sticky=W)

btnUnloadMTO = Button(window,width=15,text='Unload MTOs',command=lambda: UnloadObjs(root,'MTO',pattern=txtUnloadMTO.get()))
btnUnloadMTO.grid(column=3,row=rowTxt+1,padx=padx,pady=pady,sticky=W)

txtUnloadMTO = Entry(window,width=15)
txtUnloadMTO.delete(0, END)
txtUnloadMTO.insert(0, '*')
txtUnloadMTO.grid(column=4,row=rowTxt+1,padx=padx,pady=pady,columnspan = 1,sticky=W)

btnUnloadCon = Button(window,width=15,text='Unload Con.',command=lambda: UnloadObjs(root,'Constellation',pattern=txtUnloadCon.get()))
btnUnloadCon.grid(column=3,row=rowTxt+2,padx=padx,pady=pady,sticky=W)

txtUnloadCon = Entry(window,width=15)
txtUnloadCon.delete(0, END)
txtUnloadCon.insert(0, '*')
txtUnloadCon.grid(column=4,row=rowTxt+2,padx=padx,pady=pady,columnspan = 1,sticky=W)

def clear():
    txtBox.delete(1.0,END)
btnClear = Button(window,width=15,text='Clear TextBox',command=clear)
btnClear.grid(column=3,row=rowTxt+3,padx=padx,pady=pady,sticky=W)


# Keep window open
window.mainloop()